프로젝트: 한영 번역기 만들기
===

한국어-영어 말뭉치를 사용하여 Seq2seq 기반 번역기를 직접 만들고 구조를 이해해 본다. 또한 Attention 기법을 추가하여 성능을 높여본다. 
    
[목차]
1. 데이터 다운로드
2. 데이터 정제
3. 데이터 토큰화
4. 모델 설계
5. 훈련하기

In [2]:
# 한국어를 잘 시각화하기 위한 준비
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

In [4]:
# 라이브러리 임포트
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

print(tf.__version__)

2.4.1


# 1. 데이터 다운로드

한영 병렬 데이터를 사용합니다. 
- [jungyeul/korean-parallel-corpora](https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1)

In [1]:
!ls ~/aiffel/GoingDeeper/nlp8/*.gz

/aiffel/aiffel/GoingDeeper/nlp8/korean-english-park.train.tar.gz


# 2. 데이터 정제

1) 중복 제거    
    set 데이터형이 중복을 허용하지 않는다는 것을 활용해 중복된 데이터를 제거합니다. 데이터의 병렬 쌍이 흐트러지지 않게 주의합니다. 중복을 제거한 데이터를 cleaned_corpus 에 저장합니다.

2) 정규식 추가    
    앞서 정의한 preprocessing() 함수는 한글에서는 동작하지 않습니다. 한글에 적용할 수 있는 정규식을 추가하여 함수를 재정의합니다.

3) 토큰 추가    
    타겟 언어인 영문엔 \<start> 토큰과 \<end> 토큰을 추가하고 split() 함수를 이용하여 토큰화합니다. 한글 토큰화는 KoNLPy의 mecab 클래스를 사용합니다.
    
모든 데이터를 사용할 경우 학습에 굉장히 오랜 시간이 걸립니다. cleaned_corpus로부터 토큰의 길이가 40 이하인 데이터를 선별하여 eng_corpus와 kor_corpus를 각각 구축합니다.